In [2]:
import os
import re
import pandas as pd


rate = 'results_subseq_rate05'
for dataset in ["BeijingAir", "Electricity", "ETT_h1", "ItalyAir", "Pedestrian", "PeMS", "PhysioNet2012", "PhysioNet2019"]: 
    # Define the path to the log directory
    log_dir = f"{rate}_log/{dataset}_log"
    # List of model names
    model_names = [
        "Autoformer", "BRITS", "Crossformer", "CSDI", "DLinear", "ETSformer",
        "FiLM", "FreTS", "GPVAE", "GRUD", "Informer", "iTransformer", "Koopa",
        "MICN", "MRNN", "NonstationaryTransformer", "PatchTST", "Pyraformer",
        "SAITS", "SCINet", "StemGNN", "TimesNet", "Transformer", "USGAN"
    ]

    # Define a regex pattern to extract the required metrics
    metrics_pattern = re.compile(r"MAE=(\d+\.\d+) ± (\d+\.\d+), MSE=(\d+\.\d+) ± (\d+\.\d+), MRE=(\d+\.\d+) ± (\d+\.\d+), average inference time=(\d+\.\d+)")
    params_pattern = re.compile(r"the number of trainable parameters: ([\d,]+)")

    # Dictionary to store the results
    results = {
        "Model": [],
        "Size": [],
        "MAE": [],
        "MSE": [],
        "MRE": [],
        "Time": []
    }

    # Loop through each model's log file and extract metrics
    for model in model_names:
        file_path = os.path.join(log_dir, f"{model}_{dataset}.log")
        if os.path.exists(file_path):
            with open(file_path, 'r') as file:
                lines = file.readlines()
                params = "0"
                for line in lines:
                    if params_pattern.search(line):
                        params_match = params_pattern.search(line)
                        params = params_match.group(1)
                    if metrics_pattern.search(line):
                        metrics_match = metrics_pattern.search(line)
                        if metrics_match:
                            mae, mae_std, mse, mse_std, mre, mre_std, time = metrics_match.groups()
                            results["Model"].append(model)
                            results["Size"].append(params)
                            results["MAE"].append(f"{float(mae):.3f} ± {float(mae_std):.3f}")
                            results["MSE"].append(f"{float(mse):.3f} ± {float(mse_std):.3f}")
                            results["MRE"].append(f"{float(mre):.3f} ± {float(mre_std):.3f}")
                            results["Time"].append(time)
                            break
                else:
                    results["Model"].append(model)
                    results["Size"].append(params)
                    results["MAE"].append("0")
                    results["MSE"].append("0")
                    results["MRE"].append("0")
                    results["Time"].append("0")
        else:
            results["Model"].append(model)
            results["Size"].append("0")
            results["MAE"].append("0")
            results["MSE"].append("0")
            results["MRE"].append("0")
            results["Time"].append("0")

    # Create a DataFrame from the results
    df = pd.DataFrame(results)
    df.to_csv(f"results_subseq/{rate}_{dataset}_results.csv", index=False)


OSError: Cannot save file into a non-existent directory: 'results_01'